# NLI Example

Based on https://talman.io/nli/pytorch/demo/2020/12/11/natural-language-inference-with-pytorch-and-transformers.html

In [ ]:
!grep MemTotal /proc/meminfo


MemTotal:       13305368 kB


In [ ]:
!pip install transformers datasets
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
VERSION = "1.8.1"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

     |████████████████████████████████| 2.3MB 5.4MB/s 
     |████████████████████████████████| 245kB 26.7MB/s 
     |████████████████████████████████| 3.3MB 36.9MB/s 
     |████████████████████████████████| 901kB 12.8MB/s 
     |████████████████████████████████| 245kB 32.5MB/s 
     |████████████████████████████████| 122kB 44.4MB/s 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0    97k      0 --:--:-- --:--:-- --:--:--   97k
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.8.1 ...
     |████████████████████████████████| 61kB 3.3MB/s 
Uninstalling torch-1.8.1+cu101:
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-clie

First, load the dataset. We will load a small sample for this as the NLI dataset is really big

In [ ]:
import datasets
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = 'bert-base-uncased'
nli_data = datasets.load_dataset("multi_nli")
train_dataset = nli_data['train'].select(range(10_000))
# limiting the training set size to 20,000 for demo purposes
dev_dataset = nli_data['validation_matched'].select(range(5_000))

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


We will apply standard steps in dataset preprocessing with just a little detail: we will save preprocessed datasets to disk.

Why? Because, further on, we will run several separate processes using TPUs, and instead of all of them performing this task once per process, we do it now and then just load the preprocessed version.

Remember that here in Colab we have one core but many TPU cores. Minimize CPU usage as much as you can

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.model_max_length = 256

def tokenize(batch):
    return tokenizer(batch['premise'], batch['hypothesis'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=64)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=64)



def format_dataset(dataset):
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)



Sanity check. Have we tokenized properly?

In [ ]:
tokenizer.decode(train_dataset[0]["input_ids"])

'[CLS] conceptually cream skimming has two basic dimensions - product and geography. [SEP] product and geography are what make cream skimming work. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

Save them to disk

In [ ]:
train_path = "datasets/train/"
dev_path = "datasets/dev/"

train_dataset.save_to_disk(train_path)
dev_dataset.save_to_disk(dev_path)

Now, load model. We use an `xla` wrapper


Check https://pytorch.org/xla/release/1.8/index.html#torch_xla.distributed.xla_multiprocessing.MpModelWrapper



In [ ]:
import torch_xla.distributed.xla_multiprocessing as xmp

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.train()


WRAPPED_MODEL = xmp.MpModelWrapper(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #_, _, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")

    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        #'macro f1': macro_f1,
        'precision': precision,
        'recall': recall
    }

def train_nli(model, epochs=10, warmup_steps=200):
    """
    This contains everything that must be done to train our models
    """

    print("Loading datasets... ", end="")

    train_dataset = load_from_disk(train_path)
    dev_dataset = load_from_disk(dev_path)


    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        warmup_steps=warmup_steps,
        evaluation_strategy="epoch",
        weight_decay=0.01,
        logging_dir='./logs',
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    results = []

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
    )
    trainer.place_model_on_device = False
    trainer.train()

    trainer.save_model("nli_model/")
    tokenizer.save_pretrained("nli_model/")

We have to define a `_mp_fn` function which will be called with the index of the TPU core that it will run on.

In order to diminish TPU memory usage, we use the `WRAPPED_MODEL` defined before.

Then, we call `xmp.spawn` with `start_method='fork'`.

In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

def _mp_fn(index):
    device = xm.xla_device()
    # We wrap this
    model = WRAPPED_MODEL.to(device)

    train_nli(model)

xmp.spawn(_mp_fn, start_method="fork")

Loading datasets... 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.765537,0.674800,0.673628,0.675056,0.673518
2,No log,0.673774,0.729800,0.723677,0.731294,0.725533
3,No log,0.893317,0.740400,0.738806,0.741493,0.739322
4,0.650100,1.344917,0.747600,0.746187,0.746626,0.746342
5,0.650100,1.625435,0.746400,0.742058,0.746168,0.743085
6,0.650100,1.775800,0.750000,0.748431,0.748635,0.748507
7,0.094400,1.877627,0.748000,0.745942,0.746391,0.746097
8,0.094400,1.973956,0.747400,0.746556,0.747171,0.746683
9,0.094400,1.964262,0.748000,0.746196,0.746289,0.746270
10,0.020100,1.978223,0.747400,0.745898,0.746032,0.745960


Loading datasets... 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.765537,0.674800,0.673628,0.675056,0.673518
2,No log,0.673774,0.729800,0.723677,0.731294,0.725533
3,No log,0.893317,0.740400,0.738806,0.741493,0.739322
4,0.643400,1.344917,0.747600,0.746187,0.746626,0.746342
5,0.643400,1.625435,0.746400,0.742058,0.746168,0.743085
6,0.643400,1.775800,0.750000,0.748431,0.748635,0.748507
7,0.103000,1.877627,0.748000,0.745942,0.746391,0.746097
8,0.103000,1.973956,0.747400,0.746556,0.747171,0.746683
9,0.103000,1.964262,0.748000,0.746196,0.746289,0.746270
10,0.021900,1.978223,0.747400,0.745898,0.746032,0.745960


Loading datasets... 

...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.765537,0.674800,0.673628,0.675056,0.673518
2,No log,0.673774,0.729800,0.723677,0.731294,0.725533
3,No log,0.893317,0.740400,0.738806,0.741493,0.739322
4,0.620200,1.344917,0.747600,0.746187,0.746626,0.746342
5,0.620200,1.625435,0.746400,0.742058,0.746168,0.743085
6,0.620200,1.775800,0.750000,0.748431,0.748635,0.748507
7,0.093200,1.877627,0.748000,0.745942,0.746391,0.746097
8,0.093200,1.973956,0.747400,0.746556,0.747171,0.746683
9,0.093200,1.964262,0.748000,0.746196,0.746289,0.746270
10,0.017700,1.978223,0.747400,0.745898,0.746032,0.745960


Loading datasets... 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.765537,0.674800,0.673628,0.675056,0.673518
2,No log,0.673774,0.729800,0.723677,0.731294,0.725533
3,No log,0.893317,0.740400,0.738806,0.741493,0.739322
4,0.633200,1.344917,0.747600,0.746187,0.746626,0.746342
5,0.633200,1.625435,0.746400,0.742058,0.746168,0.743085
6,0.633200,1.775800,0.750000,0.748431,0.748635,0.748507
7,0.103500,1.877627,0.748000,0.745942,0.746391,0.746097
8,0.103500,1.973956,0.747400,0.746556,0.747171,0.746683
9,0.103500,1.964262,0.748000,0.746196,0.746289,0.746270
10,0.019900,1.978223,0.747400,0.745898,0.746032,0.745960


Loading datasets... 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.765537,0.674800,0.673628,0.675056,0.673518
2,No log,0.673774,0.729800,0.723677,0.731294,0.725533
3,No log,0.893317,0.740400,0.738806,0.741493,0.739322
4,0.640800,1.344917,0.747600,0.746187,0.746626,0.746342
5,0.640800,1.625435,0.746400,0.742058,0.746168,0.743085
6,0.640800,1.775800,0.750000,0.748431,0.748635,0.748507
7,0.113600,1.877627,0.748000,0.745942,0.746391,0.746097
8,0.113600,1.973956,0.747400,0.746556,0.747171,0.746683
9,0.113600,1.964262,0.748000,0.746196,0.746289,0.746270
10,0.024900,1.978223,0.747400,0.745898,0.746032,0.745960
